Define Hyperparameters and Paths (Then You Can Just Run All of the Cells)

In [ ]:
WELLS = ["CCS1", "VW1", "VW2"]

FEATMAP_PATH = "REU Project/Excel Sheets/Feature Map.xlsx"
LAS_PATH = "REU Project/Data/LAS Data Files"
FORMATION_PATH = "REU Project/Excel Sheets/Rock Formation Depths.xlsx"
RAW_PATH = "REU Project/Data/Raw CSV Files/raw_"
NORM_PATH = "REU Project/Data/Raw CSV Files/norm_"

Imports and Installations

In [ ]:
pip install lasio

In [ ]:
import lasio
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import warnings
import pandas as pd

Read Feat Excel File and Get All Features that Combination of Wells Share

In [ ]:
featmap_df = pd.read_excel(FEATMAP_PATH, index_col = 0, sheet_name="for code")
filtered_featmap = featmap_df[WELLS] #only consider pre-determined wells

feat_mnemonics = []
for idx, row in filtered_featmap.iterrows():
  if not ((row == 0).any() | pd.isna(row)).any():
    #keep track of features that all wells have data for
    feat_mnemonics.append(idx)

print("Number of Features:", len(feat_mnemonics))
print("Features: ")
for feat in sorted(feat_mnemonics):
  print(feat)

Number of Features: 22
Features: 
BOUND_WATER
CHLORITE
DOLOMITE
DTCO
DTSM
ILLITE
K-FELDSPAR
KAOLINITE
KSDR_PY
N-FELDSPAR
PIGE
PYRITE
QUARTZ
RHOZ
RLA2
RLA3
RLA4
RLA5
RXOZ
UWATER
XIWATER
XWATER


Read Feat Map Excel File and Get Data for Each Feature (According to Excel Sheet detailing which data file each feature should be extracted from)

In [ ]:
well_files = {} #maps each well to its relevant files/corresponding data
for well in WELLS:
  file_to_feat = {} #maps each file to the relevant feature data it contains
  files = list(set(featmap_df[well].tolist())) #get well's unique files
  files = [file for file in files if file != 0 and not pd.isna(file)]
  for file in files:
    #write las file to a pandas df
    las = lasio.read(LAS_PATH + well + "/" + file +".las")
    las_df = pd.DataFrame(las.data, columns=las.keys())

    #manual fix of known inconsistencies in data files
    las_df = las_df.rename(columns={'MD': 'DEPT'})
    las_df = las_df.rename(columns={'KSDRBB' : 'KSDR_PY'})
    las_df = las_df.rename(columns={'K_FELDSPAR' : 'K-FELDSPAR'})
    las_df = las_df.rename(columns={'N_FELDSPAR' : 'N-FELDSPAR'})

    #get feature data according for files indicated by excel sheet
    feats_from_map = featmap_df[featmap_df[well] == file].index.tolist()

    #filter so only considering features that all relevvant wells share
    shared_feats = [feat for feat in feats_from_map if feat in feat_mnemonics]
    feats = ["DEPT"] + shared_feats
    las_df = las_df[feats]

    file_to_feat[file] = las_df
  well_files[well] = file_to_feat

Ensures All Depths are Rounded to 0.5 ft

In [ ]:
for well, file_to_feat in well_files.items():
  for file, df in file_to_feat.items():
    df["DEPT"] = (df["DEPT"] * 2).round() / 2
    file_to_feat[file] = df
  well_files[well] = file_to_feat

Make One File For Each Well

In [ ]:
def pd_concat(prev_df, new_df):
  """
  Helper function that merges dataframes and takes care of duplicate features.
  """
  if prev_df.empty:
    return new_df #this is first df to be concatenated
  else:
    merged_df = pd.merge(prev_df, new_df, on='DEPT', how='outer')

    #deal with any feature duplicates (room for refinement)
    for feat in feat_mnemonics:
      if feat + "_x" in merged_df.columns and feat not in merged_df.columns:
        merged_df[feat] = \
              merged_df[f'{feat}_y'].combine_first(merged_df[f'{feat}_x'])
        merged_df = merged_df.drop([f'{feat}_x', f'{feat}_y'], axis=1)

    return merged_df


dfs = {} #maps well to singular, merged dataframe
for well, file_to_feat in well_files.items():
  df = pd.DataFrame()
  for new_df in file_to_feat.values():
    df = pd_concat(df, new_df)

    #put columns in desirable order
    cols = sorted(df.columns.to_list())
    cols.remove("DEPT")
    cols.insert(0, "DEPT")
    df = df.reindex(columns=cols)

    dfs[well] = df
  dfs[well] = df

Remove Rows that contain NaN values

In [ ]:
def remove_na():
  for well, df in dfs.items():
    init_size = df.shape[0]
    dfs[well] = df.dropna(axis=0) #remove row if row contains a NaN
    fin_size = dfs[well].shape[0]
    print(f"{init_size-fin_size} rows were removed from well {well} \
    ({init_size} -> {fin_size})")

remove_na()

0 rows were removed from well CCS1     (4217 -> 4217)
0 rows were removed from well VW1     (3999 -> 3999)
0 rows were removed from well VW2     (3413 -> 3413)


Add Rock Formation Column to Dataset

In [ ]:
def add_rock_forms(dataset, well):
  """
  Helper function that refers to pre-made excel sheet to incorporate rock
  formation at each depth into the dataset.
  """
  formation_pd = pd.read_excel(FORMATION_PATH, header=0)
  filtered_formations = formation_pd[formation_pd["Well"] == well]
  for index, row in dataset.iterrows():
    dept = row["DEPT"]
    for index2, row2 in filtered_formations.iterrows():
      min_dept = row2["Min Dept"]
      max_dept = row2["Max Dept"]
      if dept >= min_dept and dept < max_dept:
        dataset.loc[index, 'FORM'] = row2["Structure"]

  return dataset


for well, df in dfs.items():
  dfs[well] = add_rock_forms(df, well) #add formations to dataset

Make Raw CSV File For Each Well

In [ ]:
#sort data in increasing depth order for ease of interpretation
for well, df in dfs.items():
  dfs[well] = df.sort_values(by="DEPT")

#write raw data to new files
for well, df in dfs.items():
  filename = RAW_PATH + well + '.csv'
  df.to_csv(filename, index=False)

Applies Min-Max Normalization to Raw Data and Writes to CSV Files

In [ ]:
def make_norm_minmax_CSV(data_sample):
  """
  Applies min-max normalization to data column.
  """
  max, min = np.max(data_sample), np.min(data_sample)
  data_sample_normalized = (data_sample - min)/(max - min)
  return data_sample_normalized


for well, df in dfs.items():
  for col in df.columns:
    #apply norm to each column in dataframe
    if col != "DEPT" and col != "FORM": #don't normalize the depths/form
      df[col] = make_norm_minmax_CSV(df.loc[:, col])

#write normalized data to new files
for well, df in dfs.items():
  filename = NORM_PATH + well + '.csv'
  df.to_csv(filename, index=False)